# Coin Seminar Plant Sleep Quality Regression

## Preparation

First of all the required packages need to be installed.

In [ ]:
!pip install timm librosa torch torchvision tqdm torcheval cjm_pytorch_utils

Afterwards we will import all the required packages.

In [ ]:
import random
import math
import multiprocessing
from pathlib import Path
import json

import pandas as pd
import numpy as np
import os
import librosa, librosa.display
import matplotlib.pyplot as plt
%matplotlib inline

from datetime import datetime

from PIL import Image
import torchaudio
from transformers import ASTFeatureExtractor

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import timm
import torch.nn as nn
from torch.optim.lr_scheduler import StepLR, LambdaLR
import torch.optim as optim

from torch.amp import autocast
from torch.cuda.amp import GradScaler
from torcheval.metrics import R2Score
from cjm_pytorch_utils.core import  get_torch_device
from tqdm.notebook import tqdm
from google.colab import drive

In [ ]:
seed = 42
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = True

In [ ]:
# Move model to GPU if available
device = get_torch_device()

The notebook will be mounted to google drive to ensure persistent files for all sessions.

We will then select the directory, where all plant recordings and labels are saved.

In [ ]:
%cd /kaggle/input/coin-data/COIN Projekt

### Helper functions

We will use the following helper function to load and preprocess the audio data. The function will create a tuple of arrays containing the Mel Frequency Cepstral Coefficients, the values for the MEL Spectrograms, the filenames, and the associated subset.

In [ ]:
# Load and preprocess audio data
def load_and_preprocess_data(data_dir, subsets, hop_length=512, n_mels=128, n_mfcc=20, fmax = 20, n_fft=2048):
    """Loads and preprocesses audio data from specified directories.

    Args:
      data_dir: The directory containing the audio data.
      subsets: A list of subdirectories within `data_dir` to load data from.
      hop_length: The hop length to use when computing the Mel spectrogram.
      n_mels: The number of Mel frequency bins to use.

    Returns:
      A tuple containing:
        - A list of Mel spectrograms, one for each audio file.
        - A NumPy array of filenames corresponding to the Mel spectrograms.
        - A list of MFCCs, one for each audio file.
        - A list of subset names corresponding to the Mel spectrograms.
    """
    data = []
    filenames = []
    signals = []
    subset_names = []
    audio_dir = data_dir
    # We iterate through each subset
    for i, subset in enumerate(subsets):
      subset_dir = os.path.join(audio_dir, subset)
      for filename in os.listdir(subset_dir):
        try:
          if filename.endswith('.wav'):
              file_path = os.path.join(subset_dir, filename) # joining the path to the subset and the filename to get the full path
              print(file_path)
              audio_data, sample_rate = librosa.load(file_path, sr=None) # We load the audio data with librosa
              # Perform preprocessing (e.g., convert to Mel spectrogram)
              audio_signal = librosa.feature.mfcc(y=audio_data, sr=sample_rate, hop_length=hop_length, n_mfcc=n_mfcc, fmax=fmax, n_fft=n_fft) # Extracting Mel Frequency Cepstral Coefficients
              mel_spectrogram = librosa.feature.melspectrogram(y=audio_data, sr=sample_rate, n_mels=n_mels, hop_length=hop_length, fmax=fmax, n_fft=n_fft) # Extracting  Mel spectrogram
              mel_spectrogram = librosa.power_to_db(mel_spectrogram, ref=np.max) # We convert the amplitudes to decibel scale
              filenames.append(filename) # Saving the filenames in case we need it later
              subset_names.append(subset)
              signals.append(audio_signal)
              data.append(mel_spectrogram)
        except:
          pass
    return data, np.array(filenames), signals, subset_names

## Loading the data

We will load the labels separately as a Dataframe from the sleep circle data export. It is important to note that the datetime columns "Went to bed" and "Woke up" need to be converted to the datetime with the '%Y-%m-%d %H' format.

In [ ]:
sleep_circle_dir = "./data/Labels/"
sleep_circle_dir

In [ ]:
sleep_circle_data_list = []
for filename in os.listdir("./data/Labels/"):
  sleep__circle_data = pd.read_csv(os.path.join(sleep_circle_dir, filename), delimiter= ";")
  sleep__circle_data.rename(columns={"Start": "Went to bed", "End": "Woke up"}, inplace = True)
  sleep__circle_data["Went to bed"] = pd.to_datetime(pd.to_datetime(sleep__circle_data["Went to bed"]).dt.strftime('%Y-%m-%d %H'))
  sleep__circle_data["Woke up"] =  pd.to_datetime(pd.to_datetime(sleep__circle_data["Woke up"]).dt.strftime('%Y-%m-%d %H'))
  sleep__circle_data["subset_name"] = filename.split(".csv")[0]
  sleep_circle_data_list.append(sleep__circle_data)
sleep_circle_df = pd.concat(sleep_circle_data_list)
sleep_circle_df

In [ ]:
sleep_circle_df["subset_name"].unique()

With the helper function we will load and preprocess the plant recordings.

In [ ]:
hop_length=512
n_mels=30
n_mfcc=20*2
sr = 144
n_fft= 2048
mfcc = True
fmax = 20

In [ ]:
data_dir = './data/Audio_Data'
subsets = ['behrad', 'linus', 'Jasper', 'Fynn']
audio_files, filenames, signals, subset_names  = load_and_preprocess_data(data_dir = data_dir, subsets = subsets, hop_length=hop_length, n_mels=n_mels, n_mfcc=n_mfcc, fmax = fmax, n_fft = n_fft)

From the filenames we extract the timestamp and convert it to the same format as the "Went to bed" column from the sleep circle data.

In [ ]:
audio_timestamps = []
for filename in filenames:
  timestamp = filename.split("_")[-1]
  timestamp = timestamp.split(".")[0]
  timestamp = int(timestamp)
  timestamp = datetime.utcfromtimestamp(timestamp/1000).strftime('%Y-%m-%d %H')
  audio_timestamps.append(timestamp)

We join the arrays to a single dataframe for easier use.

In [ ]:
audio_df = pd.DataFrame(data = {"timestamp": audio_timestamps, "mel_spectrogram": audio_files, "mfcc": signals, "filenames": filenames, "subset": subset_names})
audio_df['timestamp'] = pd.to_datetime(audio_df['timestamp'])
audio_df

We will sort the sleep circle data and the plant recording data to merge both dataframe with eachother.

In [ ]:
sleep_circle_df = sleep_circle_df.sort_values("Went to bed")

audio_df = audio_df.sort_values('timestamp')

Because the plant recording starting time and the sleep circle timestamp sometimes do not match to the exact hour, we will merge both dataframes on the "Went to bed" column and the "timestamp" column with a range of 3 hours.

In [ ]:
concat_arr = []
for subset in subsets:
  subset_df = audio_df[audio_df["subset"] == subset]
  sleep_circle_subset_df = sleep_circle_df[sleep_circle_df["subset_name"] == subset]
  subset_merged_df = pd.merge_asof(sleep_circle_subset_df, subset_df, left_on="Went to bed", right_on ="timestamp", direction='nearest', tolerance=pd.Timedelta('3 hour'))
  concat_arr.append(subset_merged_df)
merged_df = pd.concat(concat_arr)

We drop all rows which have null values for the mel_spectrogram. Furthermore we convert the sleep quality column to the float data type ranging from 0 to 1.

In [ ]:
merged_df = merged_df.dropna(subset = ["mel_spectrogram"]).reset_index(drop=True)
merged_df["Sleep Quality"] = merged_df["Sleep Quality"].apply(lambda x: float(x.replace("%", ""))/100)
#merged_df["Sleep Quality"] = merged_df["Sleep Quality"].apply(lambda x: float(x.replace("%", "")))
merged_df

In [ ]:
merged_df[merged_df["subset"]== "behrad"]

In [ ]:
merged_df[merged_df["subset"]== "linus"]

In [ ]:
merged_df = merged_df[merged_df["Went to bed"]<= "2024-11-22"].reset_index(drop=True)
merged_df

In [ ]:
merged_df.columns

## Visualizing the data

In [ ]:
merged_df["mel_spectrogram"].shape

In [ ]:
merged_df["mel_spectrogram"][0]

In [ ]:
np.abs(merged_df["mfcc"][0])

We plot the MEL-Spectrogram and the MFCC with the librosa.display.specshow function (https://librosa.org/doc/main/generated/librosa.display.specshow.html). All values are plotted in hz units but the scale differs (MEL-Spectrogram uses mel scale while MFCC uses hz).

In [ ]:
plt.figure(figsize=(20, 5))
librosa.display.specshow(merged_df["mel_spectrogram"][0], x_axis='time',
                         y_axis='mel', sr=144, cmap= "magma", hop_length=512)
plt.colorbar(label="hz")
plt.title('Mel-Spectrogram on dB Scale', fontdict=dict(size=18))
plt.show()

In [ ]:
plt.figure(figsize=(20, 5))
librosa.display.specshow(np.abs(merged_df["mfcc"][0]), x_axis='time',
                         y_axis = "hz", sr=144, cmap= "magma")
plt.colorbar(label = "hz")
plt.title("MFCC", fontdict=dict(size=18))
plt.show()

We will also visualize the waveform to look at the peaks of amiplitudes (https://librosa.org/doc/main/generated/librosa.display.waveshow.html).

In [ ]:
plt.figure(figsize=(20, 5))
librosa.display.waveshow(merged_df["mfcc"][0], sr = 144)
plt.title("Waveplot", fontdict=dict(size=18))
plt.xlabel("Time", fontdict=dict(size=15))
plt.ylabel("Amplitude", fontdict=dict(size=15))
plt.show()

## Training Image Classification Neural Networks

## Model loading

In [ ]:
# Load a pretrained model from timm

#model = timm.create_model('resnet18', pretrained=True)
model = timm.create_model('eva02_base_patch14_448.mim_in22k_ft_in22k_in1k', pretrained=True, num_classes=1)
model

In [ ]:
model.pretrained_cfg

### Dataset

We create a custom dataframe object for the mfcc or the melspectrogram.

In [ ]:
class AudioDataset(Dataset):
    def __init__(self, audio_data, labels, transform=None):
        self.audio_data = audio_data
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.audio_data)

    def __getitem__(self, idx):
        # Load mel spectrogram
        mel_spectrogram = np.dstack([self.audio_data[idx], self.audio_data[idx], self.audio_data[idx]])

        # Apply transforms if specified
        if self.transform:
            mel_spectrogram = self.transform(mel_spectrogram)

        label = self.labels[idx]

        return mel_spectrogram, label

# Define transform to convert numpy arrays to torch tensors and normalize
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize(model.pretrained_cfg["input_size"][1:]),  # Match input size expected by the model
    transforms.Normalize(mean=model.pretrained_cfg["mean"], std=model.pretrained_cfg["std"])
])

First we split the dataframe into train and validation set.

In [ ]:
merged_df

#merged_df["Sleep Quality"] = (merged_df["Sleep Quality"]- merged_df["Sleep Quality"].abs().min())/ (merged_df["Sleep Quality"].abs().max() - merged_df["Sleep Quality"].abs().min())

train_df = merged_df.sample(frac=0.8, random_state=42)
val_df = merged_df.drop(train_df.index).reset_index(drop=True)
train_df = train_df.reset_index(drop=True)
train_df

In [ ]:
val_df

In [ ]:
train_df.merge(val_df, on=["filenames", "Sleep Quality"], how = "inner")

In [ ]:
# Set the number of worker processes for loading data. This should be the number of CPUs available.
num_workers = multiprocessing.cpu_count()#//2
batch_size = 4
# Define parameters for DataLoader
data_loader_params = {
    'batch_size': batch_size,  # Batch size for data loading
    'num_workers': num_workers,  # Number of subprocesses to use for data loading
    'persistent_workers': True,  # If True, the data loader will not shutdown the worker processes after a dataset has been consumed once. This allows to maintain the worker dataset instances alive.
    'pin_memory': 'cuda' in device,  # If True, the data loader will copy Tensors into CUDA pinned memory before returning them. Useful when using GPU.
    'pin_memory_device': device if 'cuda' in device else '',  # Specifies the device where the data should be loaded. Commonly set to use the GPU.
}

In [ ]:
# Instantiate the dataset
dataset_train = AudioDataset(audio_data=train_df["mfcc"], labels=train_df["Sleep Quality"], transform=transform)
dataset_val = AudioDataset(audio_data=val_df["mfcc"], labels=val_df["Sleep Quality"], transform=transform)

# Create a DataLoader
data_loader = DataLoader(dataset_train, **data_loader_params, shuffle=True)
data_loader_val = DataLoader(dataset_val, **data_loader_params, shuffle=False)

It is important to note that we are doing a regression task, therefore we need to change the last layer to have only one output feature.

### Model training

In [ ]:
model = model.to(device)

In [ ]:
def get_warmup_scheduler(optimizer, num_warmup_steps, total_steps):
    """
    Returns a scheduler with a linear warmup phase.

    :param optimizer: Optimizer to adjust the learning rate for.
    :param num_warmup_steps: Number of steps for the warmup phase.
    :param total_steps: Total number of training steps.
    """
    def lr_lambda(current_step):
        if current_step < num_warmup_steps:
            return float(current_step) / float(max(1, num_warmup_steps))
        return max(0.0, float(total_steps - current_step) / float(max(1, total_steps - num_warmup_steps)))

    return LambdaLR(optimizer, lr_lambda)

In [ ]:
class WarmupDecayScheduler(optim.lr_scheduler._LRScheduler):
    def __init__(self, optimizer, warmup_steps, total_steps, min_lr=0, last_epoch=-1):
        self.warmup_steps = warmup_steps
        self.total_steps = total_steps
        self.min_lr = min_lr
        super(WarmupDecayScheduler, self).__init__(optimizer, last_epoch)

    def get_lr(self):
        current_step = self.last_epoch + 1
        if current_step < self.warmup_steps:
            # Warmup phase: Linear increase
            scale = current_step / self.warmup_steps
        else:
            # Decay phase: Cosine decay
            decay_steps = self.total_steps - self.warmup_steps
            scale = max(
                0,
                0.5 * (1 + math.cos(math.pi * (current_step - self.warmup_steps) / decay_steps))
            )
        return [
            self.min_lr + (base_lr - self.min_lr) * scale
            for base_lr in self.base_lrs
        ]

In [ ]:
lr = 1e-4
epochs = 5*3
# Define loss and optimizer
criterion = nn.HuberLoss(reduction='mean', delta=0.20)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr, eps=1e-8)
metric = R2Score()
# Define warmup and total steps
total_steps = len(data_loader) * epochs
warmup_ratio = 0.1
num_warmup_steps = int(warmup_ratio * total_steps)

# Create scheduler
#lr_scheduler = get_warmup_scheduler(optimizer, num_warmup_steps, total_steps)
lr_scheduler = WarmupDecayScheduler(optimizer, num_warmup_steps, total_steps, min_lr=1e-6)

In [ ]:
step_size = 3
gamma = 0.5

# Scheduler
#scheduler = StepLR(optimizer, step_size=step_size, gamma=gamma)

The trainings loop was adapted and inspired by the tutorial from Christian Mills about [Fine-Tuning Image Classifiers with PyTorch and the timm library for Beginners](https://christianjmills.com/posts/pytorch-train-image-classifier-timm-hf-tutorial/#preparing-the-data).

In [ ]:
# Function to run a single training/validation epoch
def run_epoch(model, dataloader, optimizer, metric, lr_scheduler, device, scaler, epoch_id, criterion, is_training):
    # Set model to training mode if 'is_training' is True, else set to evaluation mode
    model.train() if is_training else model.eval()

    # Reset the performance metric
    metric.reset()
    # Initialize the average loss for the current epoch
    epoch_loss = 0
    # Initialize progress bar with total number of batches in the dataloader
    progress_bar = tqdm(total=len(dataloader), desc="Train" if is_training else "Eval")

    # Iterate over data batches
    for batch_id, (inputs, labels) in enumerate(dataloader):
        # Move inputs and labels to the specified device (e.g., GPU)
        inputs, labels = inputs.to(device), labels.to(device)


        # Forward pass
        outputs = model(inputs).squeeze()  # Get predictions
        loss = criterion(outputs, labels.float())

        # Update the performance metric
        metric.update(outputs.detach().cpu().squeeze(), labels.detach().cpu())

        # If in training mode
        if is_training:
            if scaler:
                scaler.scale(loss).backward()
                scaler.step(optimizer)
                old_scaler = scaler.get_scale()
                scaler.update()
                new_scaler = scaler.get_scale()
                if new_scaler >= old_scaler:
                    lr_scheduler.step()
            else:
                loss.backward()
                optimizer.step()
                lr_scheduler.step()
            # Zero the parameter gradients
            optimizer.zero_grad()

        loss_item = loss.item()
        epoch_loss += loss_item
        # Update progress bar
        progress_bar.set_postfix(R_2_Score=metric.compute().item(),
                                 loss=loss_item,
                                 avg_loss=epoch_loss/(batch_id+1),
                                 lr=lr_scheduler.get_last_lr()[0] if is_training else "")
        progress_bar.update()

        # If loss is NaN or infinity, stop training
        if is_training:
            stop_training_message = f"Loss is NaN or infinite at epoch {epoch_id}, batch {batch_id}. Stopping training."
            assert not math.isnan(loss_item) and math.isfinite(loss_item), stop_training_message

    progress_bar.close()
    return epoch_loss / (batch_id + 1)


# Main training loop
def train_loop(model, train_dataloader, valid_dataloader, optimizer, metric, lr_scheduler, device, epochs, criterion, use_scaler=False):
    # Initialize a gradient scaler for mixed-precision training if the device is a CUDA GPU
    scaler = GradScaler() if device.type == 'cuda' and use_scaler else None
    best_loss = float('inf')

    # Iterate over each epoch
    for epoch in tqdm(range(epochs), desc="Epochs"):
        # Run training epoch and compute training loss
        train_loss = run_epoch(model, train_dataloader, optimizer, metric, lr_scheduler, device, scaler, epoch, criterion,  is_training=True)
        # Run validation epoch and compute validation loss
        with torch.no_grad():
            valid_loss = run_epoch(model, valid_dataloader, None, metric, None, device, scaler, epoch, criterion, is_training=False)

        # If current validation loss is lower than the best one so far, save model and update best loss
        if valid_loss < best_loss:
            best_loss = valid_loss
            metric_value = metric.compute().item()
            #torch.save(model.state_dict(), checkpoint_path)

            training_metadata = {
                'epoch': epoch,
                'train_loss': train_loss,
                'valid_loss': valid_loss,
                'metric_value': metric_value,
                'learning_rate': lr_scheduler.get_last_lr()[0],
            }

            # Save best_loss and metric_value in a JSON file
            #with open(Path(checkpoint_path.parent/'training_metadata.json'), 'w') as f:
            #    json.dump(training_metadata, f)

    # If the device is a GPU, empty the cache
    if device.type != 'cpu':
        getattr(torch, device.type).empty_cache()

In [ ]:
checkpoint_path = "Nichts"

In [ ]:
train_loop(model=model,
           train_dataloader=data_loader,
           valid_dataloader=data_loader_val,
           optimizer=optimizer,
           metric=metric,
           lr_scheduler=lr_scheduler,
           device=torch.device(device),
           epochs=epochs,
           use_scaler = True,
           #checkpoint_path=checkpoint_path,
           criterion = criterion)

In [ ]:
# Ensure the model is in evaluation mode
model.eval()

# Perform inference
predictions = []
with torch.no_grad():  # Disable gradient computation for inference
    for inputs, labels in data_loader_val:
        # Move data to device
        mel_spectrogram = inputs.to(device)

        # Forward pass
        output = model(mel_spectrogram)

        # Iterate through each element in the output and label tensors and append them to the predictions list
        for out, lab in zip(output.squeeze(), labels):
            predictions.append((out.item(), lab.item()))

# Print the predictions
for i, pred in enumerate(predictions):
    print(f"Prediction for audio {i}: {pred[0]}  Actual value for audio {i}: {pred[1]}")

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error


# Separate the predicted and true values
predicted, true = zip(*predictions)

# Compute the R² score
r2 = r2_score(true, predicted)
MSE = mean_squared_error(true, predicted)
print("R² Score:", r2)
print("MSE Loss:", MSE)